In [1]:
import json
from pathlib import Path
import os
import re

In [2]:
with open("/workdir/circt-seeds/command_mapping.json", "r") as f:
    cmd_string_mapping = json.load(f)

In [3]:
def get_cmd(text):
    # for now let's only consider mlir-opt
    pattern = r"^(circt-opt) (.*?)[|\n]"
    matches = re.findall(pattern, text)
    if len(matches) < 1:
        return None
    if len(matches) > 1:
        #print(text)
        #raise ValueError("Expected at most one match")
        return None
    command, args = matches[0]
    return command, args.split()

cmd_mapping = dict()
for filename, command_strings in cmd_string_mapping.items():
    # concatenate command strings
    full_string = ""
    for cmd_str in command_strings:
        matches = re.findall(r"//\s+RUN:\s+(\S.*)", cmd_str)
        if len(matches) < 1:
            break
        if len(matches) > 1:
            raise ValueError("Expected at most one match")
        full_string += matches[0]
    full_string = full_string.replace("\\", "")
    #if "mlir-opt" in full_string:
    #    print(full_string)
    #    print("----")
    try:
        cmd_mapping[filename] = get_cmd(full_string)
    except:
        print(filename)

# filter out non mlir-opt commands
cmd_mapping = {file: cmdargs for file, cmdargs in cmd_mapping.items() if cmdargs is not None}
cmd_mapping

{'annotations-generic.mlir': ('circt-opt',
  ['-firrtl-lower-annotations',
   '-mlir-print-op-generic',
   '-split-input-file',
   '%s']),
 'test_clockgates.mlir': ('circt-opt', ['--lower-pipeline-to-hw', '%s']),
 'flatten-io.mlir': ('circt-opt', ['--hw-flatten-io', '%s']),
 'test_basic.mlir': ('circt-opt',
  ['-split-input-file', '-convert-fsm-to-sv', '%s']),
 'imcp-locations.mlir': ('circt-opt',
  ["-pass-pipeline='builtin.module(firrtl.circuit(firrtl-imconstprop))'",
   '--mlir-print-local-scope',
   '--mlir-print-debuginfo',
   '%s']),
 'remove-groups-1.mlir': ('circt-opt',
  ["-pass-pipeline='builtin.module(calyx.component(calyx-remove-groups))'",
   '%s']),
 'modulo-problems.mlir': ('circt-opt',
  ['%s',
   '-ssp-roundtrip=verifycirct-opt',
   '%s',
   '-ssp-schedule=scheduler=simplex']),
 'canonicalization-10.mlir': ('circt-opt',
  ["-canonicalize='top-down=true", "region-simplify=true'", '%s']),
 'test_struct_create.mlir': ('circt-opt', ['-lower-handshake-to-hw', '%s']),
 'gran

In [4]:
dialects = [
    "arc",
    "calyx",
    "chirrtl",
    "comb",
    "dc",
    "emit",
    "esi",
    "firrtl",
    "fsm",
    "handshake",
    "hw",
    "hwarith",
    "ibis",
    "llhd",
    "loopschedule",
    "ltl",
    "moore",
    "msft",
    "om",
    "pipeline",
    "seq",
    "sim",
    "sim",
    "ssp",
    "sv",
    "systemc",
    "verif",
]

In [5]:
dialect_associations = dict()
for cmd, args in cmd_mapping.values():
    for arg in args:
        possible_match = re.match(r"--?convert-([a-z]+)-to", arg)
        if possible_match:
            dialect = possible_match.group(1)
            if dialect not in dialect_associations:
                dialect_associations[dialect] = []
            if arg not in dialect_associations[dialect]:
                dialect_associations[dialect].append(arg)
        possible_match = re.match(r"--?([a-z]+)", arg)
        if possible_match:
            dialect = possible_match.group(1)
            if dialect in dialects:
                if dialect not in dialect_associations:
                    dialect_associations[dialect] = []
                if arg not in dialect_associations[dialect]:
                    dialect_associations[dialect].append(arg)
with open("/workdir/mlir-eval/circt/dialect-associations.json", "w") as f:
    json.dump(dialect_associations, f)

In [6]:
dialect_associations

{'firrtl': ['-firrtl-lower-annotations',
  '--firrtl-hoist-passthrough',
  '-firrtl-print-instance-graph',
  '-firrtl-lower-layers',
  '--firrtl-emit-metadata="repl-seq-mem=true',
  '-firrtl-lower-classes',
  '-firrtl-add-seqmem-ports',
  '-firrtl-resolve-traces',
  '--firrtl-extract-instances',
  '-firrtl-lower-memory',
  '--firrtl-probe-dce',
  '-firrtl-resolve-paths',
  '-firrtl-print-nla-table',
  '-firrtl-dft',
  '--firrtl-lower-xmr',
  '--firrtl-lower-types',
  '--firrtl-inject-dut-hier'],
 'hw': ['--hw-flatten-io',
  '-hw-legalize-modules',
  '--convert-hw-to-btor2',
  '-hw-cleanup',
  '--convert-hw-to-llvm',
  '--convert-hw-to-systemc',
  '--hw-eliminate-inout-ports="read-suffix=',
  '--convert-hw-to-llhd',
  '--hw-eliminate-inout-ports',
  '-hw-lower-instance-choices',
  "-hw-generator-callout='schema-name=Schema_Name",
  '-hw-memory-sim',
  '-hw-stub-external-modules',
  '-hw-print-instance-graph',
  '--hw-specialize'],
 'fsm': ['-convert-fsm-to-sv', '--fsm-print-graph', '--c